In [1]:
import os
import numpy as np
import sys
print(sys.executable)
import tensorflow as tf
from tensorflow.keras import layers, models
import import_ipynb
import sys
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path


/bin/python3.11


2025-03-13 13:40:14.814992: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741869614.880381  802513 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741869614.897465  802513 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 13:40:15.022374: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:

class Classifier:
    def __init__(self, encoder, input_shape, num_classes):
        self.encoder = encoder
        self.model = self._build_classifier(input_shape, num_classes)
    
    def _build_classifier(self, input_shape, num_classes):
        inputs = layers.Input(shape=input_shape)

        # Freeze encoder weights
        self.encoder.trainable = False

        x = self.encoder(inputs, training=False)  # Use frozen encoder
        x = layers.Dense(32, activation="relu")(x)
        x = layers.Dense(16, activation="relu")(x)
        outputs = layers.Dense(num_classes, activation="softmax")(x)

        model = models.Model(inputs, outputs)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                    loss="sparse_categorical_crossentropy",
                    metrics=["accuracy"])
        return model


    def train(self, x_train, y_train, batch_size=64, epochs=50, validation_split=0.1):
        history = self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split)
        return history
    
    def evaluate(self, x_test, y_test):
        return self.model.evaluate(x_test, y_test)
    
    def predict(self, x):
        return self.model.predict(x)

# # Load the trained encoder from MaskedAutoencoder
# autoencoder = MaskedAutoencoder(input_dim=500)  # Use the correct input dimension

# # Define an input layer matching the expected shape
# input_layer = layers.Input(shape=(500,))  # Make sure this matches your signal window size

# # Pass the input through the autoencoder's encoder
# encoded_output = autoencoder.encoder(input_layer)

# # Create a new model for the encoder
# encoder = models.Model(input_layer, encoded_output)

encoder_path = "../models/encoder_model.keras"

if os.path.exists(encoder_path):
    # Load saved encoder
    print("Loading saved encoder...")
    encoder = tf.keras.models.load_model(encoder_path)
else:
    # Train autoencoder and extract encoder
    autoencoder = MaskedAutoencoder(input_dim=500)
    input_layer = layers.Input(shape=(500,))
    encoded_output = autoencoder.encoder(input_layer)
    encoder = models.Model(input_layer, encoded_output)

    # Save encoder
    print("Saving trained encoder...")
    encoder.save(encoder_path)



# Load the target dataset
save_dir = os.path.join(univariate_data_path, "target_univariate.npy")
target_data = np.load(save_dir, allow_pickle=True)

# Apply sliding window approach
def create_windows(data, window_size=500, step=250):
    all_windows, labels = [], []
    for sample in data:
        signal, label = sample["signal"], sample["preterm"]  # Assuming label is available
        for start in range(0, len(signal) - window_size + 1, step):
            window = signal[start : start + window_size]
            all_windows.append(window)
            labels.append(label)
    return np.array(all_windows), np.array(labels)

x_train, y_train = create_windows(target_data)

# Initialize and train the classifier
num_classes = len(set(y_train))  # Get number of unique classes
classifier = Classifier(encoder, input_shape=(500,), num_classes=num_classes)
classifier.train(x_train, y_train)


Loading saved encoder...


2025-03-13 13:41:08.643393: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


ValueError: Invalid dtype: object

In [2]:
classifier.save("classifier_model.keras")  # Saves in Keras format
print('Saved encoder!')

NameError: name 'classifier' is not defined